In [2]:
import pandas as pd
import numpy as np
import konlpy
from konlpy.tag import Twitter
import nltk
from gensim import models
import re
import requests
import pickle

In [2]:
data = None
for i in range(19):
    if data is None:
        data = pd.read_csv('data/crime{}.csv'.format((i+1)*1000))
    else:
        add_data = pd.read_csv('data/crime{}.csv'.format((i+1)*1000))
        data = data.append(add_data, ignore_index=True)
data = data.drop('Unnamed: 0', 1)       
data = data.set_index('prncYd')
data.shape

(18189, 5)

In [3]:
grouped = data.groupby('사건명').count().reset_index()
grouped.sort_values('참조조문', ascending=False)[:100]

,사건명,참조조문,판결요지,판례내용,판시사항
2517,사기,351,285,383,355
7051,폭력행위등처벌에관한법률위반,231,233,236,235
1035,교통사고처리특례법위반,170,160,182,170
7616,횡령,156,131,173,159
896,공직선거및선거부정방지법위반,135,113,143,136
1986,배임,126,119,146,132
1817,무고,122,112,133,125
1377,근로기준법위반,119,96,140,120
964,관세법위반,117,118,135,124
919,공직선거법위반,117,73,182,117


In [3]:
keywords = {
    1: ['내란'],
    2: ['외환', '간첩'],
    3: ['국기', '국장'],
    4: ['국교', '외국원수', '외국사절'], 
    5: ['범죄단체', '소요'], 
    6: ['폭발물'],
    7: ['공무원', '뇌물', '직무유기', '수뢰', '공무상비밀' , '선거방해'], 
    8: ['공무방해', '공무집행방해' '공무상표시'],
    9: ['도주', '은닉', '도피'],
    10: ['위증', '증거인멸'],
    11: ['무고'],
    12: ['신앙', '사체'],
    13: ['방화', '실화'],
    14: ['일수', '수리'],
    15: ['교통방해', '교통 방해'],
    16: ['음용수'],
    17: ['아편'],
    18: ['통화'],
    19: ['유가증권', '우표', '인지'],
    20: ['문서', '공정증서'],
    21: ['인장'],
    22: ['간통', '공연음란'],
    23: ['도박', '복표'],
    24: ['살인'],
    25: ['상해','폭행', '상해치사', '상해치상'],
    26: ['과실치상', '과실치사'],
    27: ['낙태'],
    28: ['유기', '학대'],
    29: ['체포', '감금'],
    30: ['협박'], 
    31: ['약취', '유인', '인신매매'],
    32: ['강간', '추행'],
    33: ['명예', '모욕'],
    34: ['신용', '업무방해', '경매', '입찰'],
    35: ['비밀침해', '비밀누설'],
    36: ['주거침입', '강제집행면탈'],
    37: ['권리행사'],
    38: ['절도','강도', '강도상해', '강도살인'],
    39: ['사기','공갈', '부당이득'],
    40: ['횡령','배임'],
    41: ['장물'],
    42: ['손괴']
}
data_label = data.copy()

In [4]:
for label in range(42):
    data_label[label + 1] = 0

for index, row in data_label.iterrows():
    print(index)
    for label in range(42):
        for keyword in keywords[label+1]:
            if keyword in str(row.사건명):
                data_label.loc[index, label+1] = 1
                print(keyword)

73888
67312
67316
67151
과실치사
67258
횡령
67135
사기
67223
과실치사
67256
71101
살인
67308
67309
67311
67148
도피
문서
사기
횡령
배임
68369
과실치사
67248
문서
공정증서
사기
배임
67252
67231
사기
67342
67250
67230
67228
67307
강간
강도
67180
67189
폭행
주거침입
67239
67210
67207
67211
사체
살인
유기
67209
강간
67287
67492
67494
67363
67382
도주
67362
업무방해
67493
67504
67428
67717
67769
사기
67771
문서
사기
67513
67510
문서
67514
142159
67496
67381
문서
절도
사기
횡령
67318
사기
67378
배임
67527
강간
67332
도피
살인
공갈
67526
도주
67497
유가증권
사기
67498
강간
67500
177818
배임
67415
67482
뇌물
67601
협박
강간
주거침입
절도
강도
67502
67419
유가증권
문서
신용
사기
배임
67483
68000
67810
67892
67797
68113
주거침입
67813
과실치사
112488
무고
159791
사기
159792
159817
67925
67798
67816
68051
뇌물
67803
도주
68148
67872
공무원
67874
67815
68006
상해
손괴
146480
68054
68055
공갈
68681
업무방해
67929
은닉
사체
살인
강간
68079
횡령
68080
살인
68007
96873
횡령
67804
강간
67920
67786
68137
횡령
68139
상해
강도
강도상해
68104
사기
67754
67756
사기
76895
67795
횡령
67924
유가증권
문서
사기
68112
절도
68115
공무원
67947
업무방해
68043
67618
간통
67853
67891
67811
68002
사기
74778
폭행
68146
뇌물
횡령
배임
6

강간
72258
살인
72259
72261
72262
72263
살인
72265
강간
72885
101769
강간
73352
명예
공갈
손괴
73611
문서
73612
강간
73339
73340
문서
73342
손괴
73343
73344
뇌물
문서
공정증서
73349
73353
살인
강도
강도살인
73610
과실치상
73387
73388
상해
상해치사
73377
공정증서
인장
73615
간첩
73616
73341
73354
장물
73355
방화
문서
주거침입
절도
사기
73356
강간
73358
상해
강도
강도상해
73613
절도
73389
살인
73390
73391
문서
73393
문서
횡령
73394
73378
낙태
153516
문서
73614
73110
강간
73112
문서
73117
살인
73119
72883
129690
강간
강도
73104
73103
주거침입
절도
장물
73113
72845
73109
72884
살인
73095
문서
73134
111870
72847
도피
절도
73102
폭행
협박
73105
강간
72886
73120
73121
73123
73124
73107
상해
73128
절도
73129
간첩
73130
73098
73106
살인
73131
73108
강간
73132
살인
73135
과실치사
73101
72895
문서
73116
무고
72858
방화
72859
사기
72704
72869
72871
72872
문서
72874
문서
72727
72873
문서
73096
폭발물
살인
73097
72703
72860
72705
72708
강간
68730
73586
강간
73836
73570
살인
73588
강간
73589
73602
73603
73571
73590
73592
살인
상해
상해치사
유기
73594
상해
손괴
73873
73882
문서
절도
73883
상해
73877
강간
73801
은닉
76900
사기
73802
살인
73816
73818
직무유기
유기
73819
도피
도박
강도
75320
횡령
73845
사기
장물
7357

78569
도박
사기
78570
78573
사기
78575
뇌물
문서
78272
77930
무고
문서
공정증서
사기
횡령
배임
78441
상해
78356
과실치상
78357
협박
명예
업무방해
절도
공갈
78358
78359
78501
152852
78403
문서
사기
횡령
78404
77929
무고
협박
공갈
횡령
배임
78633
문서
78509
78282
78576
사기
횡령
77603
업무방해
77812
주거침입
절도
77815
문서
주거침입
사기
77600
배임
77601
77811
78689
상해
폭행
감금
협박
명예
업무방해
주거침입
손괴
78690
절도
손괴
78246
사기
78247
배임
78248
78249
뇌물
사기
78812
78814
78024
절도
78025
77172
77371
뇌물
사기
77251
77252
상해
77420
77423
77432
뇌물
77184
78173
78187
78188
78189
뇌물
77181
배임
77418
사기
77419
77421
뇌물
77422
77825
문서
사기
78000
상해
78001
뇌물
위증
사기
78162
배임
77624
추행
77654
문서
공정증서
횡령
배임
77655
공정증서
횡령
148176
148178
도박
사기
148179
77988
공정증서
배임
77989
77990
77991
상해
77992
77993
뇌물
위증
문서
사기
77994
유가증권
문서
78164
78165
78166
78167
78168
무고
강간
78169
77661
문서
공정증서
횡령
배임
148145
148146
도박
사기
148149
배임
77510
77511
77512
77513
강간
77514
도주
77515
살인
폭행
감금
78049
입찰
배임
77175
77509
문서
사기
77254
사기
112832
업무방해
77246
무고
명예
77247
77248
77249
절도
배임
손괴
77369
유가증권
사기
77370
124862
도피
86208
문서
77680
신용
77681
뇌물
78163
문서
사

사기
81970
사기
81965
도주
81859
81778
81730
강간
80795
81817
사기
126282
무고
횡령
81641
81776
81781
뇌물
81816
폭행
업무방해
주거침입
손괴
81647
81773
문서
횡령
81780
횡령
81782
사체
살인
유기
81645
81774
뇌물
무고
81691
배임
81779
배임
81784
81822
과실치사
81775
81694
81689
문서
179332
179334
횡령
179398
81734
81692
사기
81860
사기
81961
82018
126297
통화
82121
82189
126505
82181
뇌물
82116
무고
82190
상해
강도
강도상해
82177
횡령
82174
사기
126358
상해
강도
강도상해
81973
공갈
82017
81960
165831
무고
공정증서
사기
82118
81892
81975
82074
81976
126280
81963
사기
82020
뇌물
은닉
82016
81865
82080
살인
강간
절도
64556
무고
82075
뇌물
81972
사기
81966
67511
배임
81863
81896
81967
도주
82280
82389
약취
절도
강도
82349
82390
82595
도피
사기
82596
상해
사기
82382
문서
126500
82447
사기
82344
82345
사기
82342
82589
82385
상해
82279
절도
82228
82315
사기
126315
82314
배임
98847
82124
82178
횡령
126346
82276
126180
상해
176542
은닉
사기
176544
176546
176548
교통방해
상해
업무방해
손괴
82281
모욕
82312
폭행
협박
명예
82320
82227
명예
공갈
126287
73813
간첩
82021
강제집행면탈
사기
82185
문서
82175
뇌물
횡령
82581
배임
82582
82654
82652
126448
문서
절도
사기
횡령
82527
82687
상해
주거침입
강도
강도상해
824

86256
문서
사기
86357
사기
86358
실화
86359
절도
85976
명예
85977
명예
85978
절도
85979
도주
85980
무고
85981
살인
86360
횡령
125997
업무방해
86261
86363
과실치상
86113
85982
사체
살인
유기
93199
범죄단체
85983
횡령
85984
86141
살인
강도
강도살인
165976
공무원
86504
폭행
86746
절도
86794
주거침입
절도
167386
살인
86796
문서
사기
86750
공무상비밀
권리행사
배임
86785
문서
86749
절도
168596
167523
무고
사기
86736
절도
119472
도주
86738
86786
문서
86787
공정증서
168775
횡령
배임
165777
사기
86752
직무유기
유기
절도
169472
86727
횡령
86729
86741
간첩
86742
86948
횡령
86503
뇌물
수뢰
문서
86743
간첩
167319
강간
86631
살인
167359
상해
86779
86744
횡령
86791
간첩
86950
간첩
아편
문서
86951
165982
사기
배임
165986
86780
문서
사기
86792
간첩
86757
86735
명예
업무방해
86624
입찰
86802
살인
168195
상해
절도
86772
문서
사기
86773
절도
86774
86755
86611
과실치사
86612
사기
86499
86625
문서
86760
86761
사체
상해
상해치사
유기
86725
사체
유기
86726
사기
118489
86777
문서
86728
86614
실화
86615
86619
86500
도박
86628
권리행사
86799
무고
86765
공갈
86768
상해
강도
강도상해
86769
86730
86782
수뢰
86731
사기
86506
간첩
살인
86618
살인
86620
위증
86797
공정증서
횡령
86754
86784
절도
86770
86622
과실치사
86623
업무방해
86767
86732
횡령
86781
수뢰
86758
상

...

140503
67755
강제집행면탈
107357
97288
사기
84777
무고
협박
공갈
횡령
배임
152837
낙태
99101
뇌물
94986
뇌물
배임
65008
강제집행면탈
68557
상해
업무방해
128015
입찰
권리행사
횡령
배임
93369
업무방해
114349
공정증서
69486
횡령
140504
72721
직무유기
유기
157599
도주
94382
67997
신용
배임
82179
67945
업무방해
68448
절도
98841
명예
모욕
67163
85828
무고
122003
107235
96503
104507
뇌물
횡령
121065
165576
155210
83025
사기
108734
68603
사기
95938
절도
128052
명예
122004
106564
144160
도주
99112
167764
162431
118809
강도
92364
폭행
178025
111668
126665
강도
102462
68078
102119
93424
사기
111082
156102
횡령
135787
명예
67970
명예
업무방해
횡령
손괴
64717
공갈
159802
104068
140464
은닉
사체
살인
상해
상해치사
약취
유인
강간
추행
103640
강간
68373
64834
85219
96596
뇌물
94291
92620
문서
공정증서
사기
84413
공무원
64927
도주
111589
문서
공정증서
사기
횡령
배임
163695
107674
158756
69175
126718
문서
장물
111405
111075
입찰
113069
68948
99639
154735
사기
147811
123247
69421
배임
95934
123004
무고
문서
사기
96274
102949
70675
94919
114425
도주
84297
68031
159465
85783
배임
97605
뇌물
문서
사기
68866
문서
횡령
배임
131329
84501
도피
사기
횡령
배임
108644
무고
103088
124302
122000
165018
104367
68915
67785
뇌

107860
108172
74148
과실치상
102953
112907
살인
111465
75467
도피
강도
107778
73604
살인
강간
109538
68728
153533
간첩
100821
73840
153826
92948
범죄단체
절도
154614
120337
102665
강도
178237
절도
178136
횡령
107221
173344
교통방해
70560
92075
살인
118728
110506
178326
178324
178325
178331
횡령
178378
178379
강간
주거침입
178382
80069
사기
123899
67764
업무방해
79175
도박
124949
155523
109673
140874
99310
157934
84411
상해
절도
강도
강도상해
123956
업무방해
106921
강간
92675
간통
108377
100084
뇌물
72028
138156
98540
108392
97708
70821
사기
105614
문서
횡령
107960
101207
86747
간첩
171291
70933
123245
73118
92421
98554
위증
111731
무고
간통
98053
101057
152447
99684
100791
165005
살인
111864
95505
97820
122295
사기
101916
95254
과실치상
92031
107426
상해
강도
강도상해
67850
사기
97806
주거침입
108651
상해
96961
68451
111401
사체
살인
유기
강간
122086
복표
107267
사체
살인
97670
간통
119744
84873
82888
105963
복표
99555
85250
상해
104189
입찰
94012
179274
상해
폭행
협박
모욕
108176
상해
절도
강도
강도상해
107339
과실치상
94355
118654
사기
156207
주거침입
절도
강도
68531
업무방해
118387
136369
문서
179466
106319
153827
97799
횡령
배임
86745
문서
102573
97009

공무원
직무유기
유기
82689
82343
문서
사기
횡령
82690
문서
권리행사
81964
83097
83724
81648
110793
횡령
81959
82384
절도
강도
82119
116409
67343
사기
횡령
67317
74386
83174
82117
83148
사기
81636
횡령
83408
82448
80528
사기
99317
81864
문서
사기
횡령
81815
문서
명예
80793
상해
82794
126656
83671
81894
상해
강간
절도
강도
강도상해
83049
116406
횡령
83676
178586
폭행
사기
67310
82492
82887
83312
83696
69974
문서
사기
83429
폭행
명예
업무방해
횡령
83388
79470
증거인멸
장물
82864
뇌물
81867
80836
117643
간통
80923
142116
살인
강간
71565
108385
108957
복표
75998
106101
사기
105696
절도
70723
79141
도주
71251
112545
114165
뇌물
횡령
83310
94081
113688
81215
76265
간첩
114106
위증
절도
배임
117892
사기
81640
모욕
79744
179292
109015
절도
98698
148084
강제집행면탈
사기
배임
173630
사기
69813
상해
82391
도피
98774
명예
76702
사기
114852
상해
강도
108109
사기
99641
살인
107234
82597
122294
173650
살인
유기
감금
173560
106113
뇌물
114422
115438
문서
사기
83828
명예
76095
73125
117371
뇌물
113258
81818
67617
상해
폭행
협박
업무방해
주거침입
손괴
117650
배임
82771
절도
강도
156680
113256
70048
무고
문서
119741
178865
사기
107080
사기
67276
114855
67495
114254
협박
공갈
80967
명예
69881
상해
79621


182241
주거침입
179910
181801
181886
177638
179782
장물
181764
181771
181772
181774
부당이득
182080
상해
폭행
손괴
177636
179902
181832
182242
상해
폭행
협박
업무방해
권리행사
사기
공갈
횡령
182243
뇌물
182244
문서
사기
배임
182245
182246
모욕
182247
횡령
배임
182249
182250
횡령
181851
181765
177305
도박
공갈
179831
사기
181817
181819
181791
181852
181857
문서
181754
181755
181773
181777
180711
과실치사
180712
공정증서
180976
180977
사기
180714
180715
180735
180777
180993
181035
공무상비밀
비밀누설
179947
179781
횡령
180690
180674
180687
180689
180778
주거침입
절도
180779
문서
사기
177511
180967
180878
사체
도박
유기
180893
180713
무고
180978
무고
사기
횡령
배임
180979
무고
180716
살인
강도
강도살인
180717
180780
주거침입
절도
180785
횡령
180786
180994
181036
뇌물
수뢰
사기
공갈
181037
181087
명예
업무방해
181063
범죄단체
180675
180691
위증
문서
사기
180694
180756
배임
179787
절도
180855
간통
180856
횡령
180905
180906
180895
상해
절도
강도
강도상해
181014
명예
180980
강간
180718
배임
180787
공정증서
강제집행면탈
180788
주거침입
절도
180789
손괴
180995
181038
문서
사기
횡령
181058
과실치사
179809
횡령
181067
도피
180676
180781
사기
180783
문서
180836
180879
공갈
손괴
180896
상해
상해치사
179571
위증
상해


183601
182506
183600
182538
폭행
추행
184802
폭행
185370
183637
182507
업무방해
182876
문서
업무방해
182877
횡령
183602
184829
명예
182508
폭행
협박
185129
명예
185416
182853
횡령
182854
사기
182857
유가증권
문서
도박
182878
은닉
문서
횡령
배임
185166
184805
살인
손괴
185366
185363
위증
사기
횡령
배임
182858
문서
사기
182859
182881
상해
185167
185171
뇌물
문서
사기
횡령
184806
손괴
185367
주거침입
절도
185254
비밀누설
배임
185256
상해
폭행
업무방해
공갈
손괴
182860
배임
182882
입찰
185175
횡령
185176
183408
185255
182883
모욕
184783
185364
185380
과실치사
학대
185186
185187
도박
184788
184816
문서
배임
185381
185257
185153
학대
185189
185191
횡령
185200
공무원
185202
공무원
184789
절도
184787
185411
배임
182865
185154
183656
183022
문서
사기
184817
권리행사
184840
추행
182866
185155
183657
185270
뇌물
공무상비밀
은닉
비밀누설
184818
184841
업무방해
184842
권리행사
182867
182868
사기
배임
185156
183658
배임
185271
사기
184796
도주
상해
184843
상해
184856
185414
183659
183661
폭행
모욕
업무방해
184742
182933
182934
살인
학대
184675
182581
사기
184482
체포
권리행사
184764
184521
183480
비밀누설
183481
사기
182935
사기
182582
182583
184483
문서
횡령
배임
184765
무고
184522
183482
182937
협박
184523
사

In [7]:
# for index, row in data_label.iterrows():
#     print(index)
#     if  '위반' in str(row.사건명):
#         for label in range(42):
#                 data_label.loc[index, label+1] = 0

73888
67312
67316
67151
67258
67135
67223
67256
71101
67308
67309
67311
67148
68369
67248
67252
67231
67342
67250
67230
67228
67307
67180
67189
67239
67210
67207
67211
67209
67287
67492
67494
67363
67382
67362
67493
67504
67428
67717
67769
67771
67513
67510
67514
142159
67496
67381
67318
67378
67527
67332
67526
67497
67498
67500
177818
67415
67482
67601
67502
67419
67483
68000
67810
67892
67797
68113
67813
112488
159791
159792
159817
67925
67798
67816
68051
67803
68148
67872
67874
67815
68006
146480
68054
68055
68681
67929
68079
68080
68007
96873
67804
67920
67786
68137
68139
68104
67754
67756
76895
67795
67924
68112
68115
67947
68043
67618
67853
67891
67811
68002
74778
68146
67805
67921
67700
67702
67549
67551
67552
67944
67674
67675
67729
67826
67677
67721
67676
67722
67768
67753
67907
67614
67615
67697
67751
67946
67723
67660
67724
67752
67909
67869
67950
67598
67682
67639
67577
67579
67642
67643
67727
67666
67758
67757
67760
67701
67596
67726
67728
67730
67829
67870
67767
67578
680

75339
75340
75090
75091
75092
75059
75052
75060
75061
75062
123852
75073
75075
75342
75343
101902
75331
75088
75076
67257
75047
75050
75051
75071
75058
75468
75469
75017
67147
177767
75043
75045
75046
70883
74794
75021
75024
118647
75025
75027
74803
75031
75032
74801
74808
75015
75028
75029
75034
74784
74795
75030
74804
74805
74806
75033
74807
75056
177822
75078
75035
75036
75037
75042
75044
74790
74791
74800
74799
75077
75080
75039
75040
74788
74792
74793
74786
75041
75020
75022
75023
74787
75886
75788
75680
75679
75779
75780
75681
75791
75873
75782
75783
73835
75790
75682
75874
75875
75675
75683
75792
75684
76078
75781
99798
76079
76080
75877
75878
75670
75671
75301
75692
75880
100815
75786
75676
75678
75881
75882
75883
75884
75885
75872
75456
173557
173558
173559
75887
75888
75889
75300
75890
75302
75458
75672
75673
75674
75312
75677
76082
76083
75997
75462
75322
75323
75324
75326
75327
75294
75451
75452
75453
75295
75455
75328
74158
75307
75325
75329
75330
75440
75454
75457
75296
7

81295
81104
81058
81188
81358
81360
81137
81324
81542
81594
81637
177851
81696
81688
81642
81549
177852
81551
81495
81494
81404
81498
81544
81554
81497
81363
81545
81547
81399
81491
81400
81402
179387
81546
81403
81541
81398
81548
81595
81288
81292
81731
177772
81644
177774
81968
81970
81965
81859
81778
81730
80795
81817
126282
81641
81776
81781
81816
81647
81773
81780
81782
81645
81774
81691
81779
81784
81822
81775
81694
81689
179332
179334
179398
81734
81692
81860
81961
82018
126297
82121
82189
126505
82181
82116
82190
82177
82174
126358
81973
82017
81960
165831
82118
81892
81975
82074
81976
126280
81963
82020
82016
81865
82080
64556
82075
81972
81966
67511
81863
81896
81967
82280
82389
82349
82390
82595
82596
82382
126500
82447
82344
82345
82342
82589
82385
82279
82228
82315
126315
82314
98847
82124
82178
126346
82276
126180
176542
176544
176546
176548
82281
82312
82320
82227
126287
73813
82021
82185
82175
82581
82582
82654
82652
126448
82527
82687
82484
82483
82583
82795
82489
8269

69283
170643
170692
170707
170708
75018
170412
87658
168619
170834
167323
168624
87772
170405
170437
170452
85909
166731
171749
172055
155897
170916
167295
87510
87511
167770
167772
172140
172166
172168
172170
171252
153855
152536
153351
152838
73815
152541
152871
154506
153110
154744
154746
154147
152856
154748
154753
152862
154525
154751
152428
153628
82120
158292
152509
153624
167848
165878
167329
167333
153136
87894
167431
167334
145539
88002
153633
167266
87898
153634
153637
153638
153651
152613
168098
153630
168131
167653
153641
74180
154031
87892
167304
167307
167309
167914
153640
154983
153832
154992
155004
155009
142617
154977
154976
141184
154377
154380
155007
155025
153327
155023
155010
88601
154589
154451
154518
154621
155038
155041
154514
155032
152569
154001
154015
153512
125175
154149
153071
152830
76690
154142
154144
156591
153532
70932
154982
154989
157994
154991
155015
154980
156592
155001
155000
155019
154403
156568
156577
154967
155013
154017
153833
101569
158057
15

98289
98288
97997
98071
97059
173239
173240
97289
97341
97611
97998
97932
97777
97518
97519
97805
98250
98283
97224
96759
96764
97290
97764
97766
97767
98073
97933
97830
97458
97621
97223
98075
98133
97622
97827
107957
97807
96756
97220
96763
98001
177834
98233
97704
97620
96883
97811
97829
97861
97862
98235
97994
97623
97005
97006
97003
96999
97525
97526
97802
98251
97930
97824
97863
97773
98009
97104
97105
68765
97627
98072
97931
97535
97536
97537
97776
86119
96832
97601
97172
97412
97408
97410
98083
98134
98284
97598
111462
97664
97667
97709
97173
97103
97447
98008
98285
98070
97825
97276
97274
97669
97624
97625
97444
97446
98007
98131
97826
97928
97278
97271
97272
97783
97267
97990
97991
97671
97705
97521
97524
98082
97929
97999
97599
97531
97355
97356
97399
97397
97398
97468
84872
97273
97778
97265
97262
86117
97626
97800
98067
98068
98162
97357
97532
97533
97549
97546
97547
97367
98239
97360
97097
97619
97657
97660
96294
97053
97610
97616
97465
97466
97226
96934
97060
97658
97659

...

159884
161654
158251
161251
160412
160419
160976
160980
161560
160602
161143
159934
159938
161211
160716
161503
161504
161731
161166
162283
160421
160653
161518
145243
158528
160983
156062
156253
73832
155736
161621
159805
155708
161529
156070
124303
158734
155710
170594
166422
166501
170644
171598
165193
166424
166503
166507
171211
166478
168357
167596
167597
169735
169733
169745
169747
169749
170413
170415
170881
171370
171417
169074
169090
168459
167603
169743
166453
164699
167464
167468
165911
172442
164291
163892
163871
169783
164073
165202
167600
170191
167450
165924
165431
172448
165873
169492
163831
170952
169203
171671
163894
163883
163884
165879
168344
172447
164639
165836
163886
107970
164920
165880
172458
164760
166518
171626
169594
168594
164913
172500
171060
171369
171707
163887
168960
170595
168606
172454
164780
167593
167594
166325
164836
171628
171708
171732
171733
168978
172411
172417
172419
107453
164768
170686
167952
167954
167960
167964
167999
169358
168980
168174


126254
84594
171945
110802
103852
105423
86762
70530
119217
67380
149516
64929
143105
68207
111407
146253
146252
149184
142097
149879
107083
160400
70443
171820
69091
159926
97402
94532
93463
94009
85708
102514
102575
107433
104420
98415
145736
150837
157620
96455
109870
160219
160131
111868
146225
179399
179454
172412
69304
161715
124971
108460
140126
84377
170454
97935
144456
145123
84296
171337
171640
147396
146249
70674
112406
165751
143340
158450
70050
105362
172413
138238
141526
79088
85526
100873
85555
150722
141384
143960
139224
82278
97114
107436
111925
143683
81189
171059
147397
119071
114626
69055
84382
100687
70589
110990
143626
147336
173214
159837
108959
134212
76356
165757
146228
170453
143591
115884
64960
161219
165174
143593
157102
84988
68524
67759
178115
145127
142287
167512
92953
160243
108242
110691
128016
110507
97352
150569
161051
124996
97804
136146
70653
95223
97358
163739
84232
100631
178135
143826
84122
117198
99793
171785
145492
166330
98558
80190
136165
159

107860
108172
74148
102953
112907
111465
75467
107778
73604
109538
68728
153533
100821
73840
153826
92948
154614
120337
102665
178237
178136
107221
173344
70560
92075
118728
110506
178326
178324
178325
178331
178378
178379
178382
80069
123899
67764
79175
124949
155523
109673
140874
99310
157934
84411
123956
106921
92675
108377
100084
72028
138156
98540
108392
97708
70821
105614
107960
101207
86747
171291
70933
123245
73118
92421
98554
111731
98053
101057
152447
99684
100791
165005
111864
95505
97820
122295
101916
95254
92031
107426
67850
97806
108651
96961
68451
111401
122086
107267
97670
119744
84873
82888
105963
99555
85250
104189
94012
179274
108176
107339
94355
118654
156207
68531
118387
136369
179466
106319
153827
97799
86745
102573
97009
106919
94606
91418
97342
73115
97765
112165
155020
124519
73844
154011
104930
95089
97539
148073
179331
70882
106563
107143
168031
106926
124969
85892
68680
68021
106221
124524
95511
98832
124964
95386
92274
105889
179265
99034
118238
129613
9751

80865
179197
79740
82657
79284
118155
67661
179215
127087
98422
116598
75693
67261
115805
80298
114914
109189
80797
97866
105747
97050
156845
71249
127020
178863
179115
79332
67470
67469
79708
79891
113102
67229
69905
179199
157363
178866
79940
80663
169111
173637
117486
80754
80479
67224
79432
82740
69834
111078
71534
70151
79327
99683
83235
79322
74783
80718
79474
173626
73852
79469
67198
67681
112403
80659
114857
173577
173580
79391
123494
178874
174848
79524
81490
153011
94567
175218
113838
175221
175224
175399
107963
160393
175472
175405
69592
68759
174741
174746
175474
84076
80890
115176
116530
174134
174849
174858
174807
174811
156368
174658
112544
174107
98842
113399
174668
174715
174716
153990
82719
175152
175153
175158
114245
174748
174752
175554
100794
133579
117838
68391
115174
175471
113245
163501
174133
174794
174804
113754
175632
67427
93140
105207
173820
173831
174135
112908
174996
117888
174108
173767
175119
117020
174913
173808
173850
123504
100993
120016
174082
11769

181557
181627
181630
181301
181487
79622
181246
181350
181353
181362
181396
181422
181559
181560
181561
181562
181631
181632
181635
181303
181683
181684
180321
180776
180657
179486
180468
180492
179956
180603
180432
180433
180434
180190
180191
180192
180549
180552
180293
179915
180469
180587
180775
177490
180604
180435
180193
180194
180324
180557
180558
180559
180294
180658
179935
180470
180471
180588
180176
180177
180178
180626
180628
180541
180544
180195
180196
180073
180325
180661
180589
180353
180472
180590
180591
180179
180180
180611
180536
180538
180523
180197
180326
180636
180473
180474
180475
180476
180181
180612
180540
180543
180524
180198
180327
180299
180477
180478
180479
180182
180614
180615
180526
180199
180200
177591
180592
180613
180187
180616
180630
180537
180201
180248
180246
177558
177557
180480
180481
180482
180607
180188
180618
180619
180216
180247
177618
180609
180189
180534
180533
180204
180217
180623
180483
180484
180610
180597
180620
180535
180539
179388
179424


In [5]:
total = []
for i in range(42):
    print(data_label[data_label[i+1] == 1].shape[0])
    total.append(data_label[data_label[i+1] == 1].shape[0])
sum(total)
# ref_crime_data = ref_data[ref_data.apply(lambda row: '형법' in row[3], axis = 1)]
# data[data.apply(lambda row: row['참조조문'].find('형법').any(), axis=1)]
# df.apply(lambda row: row.astype(str).str.contains('TEST').any(), axis=1)
# ref_crime_data

13
121
1
0
23
9
928
6
445
286
381
77
106
4
86
0
3
15
141
1354
12
139
119
521
1134
328
13
217
98
154
63
791
385
575
42
354
98
1480
2170
2141
138
201


15172

In [6]:
data_ref = data_label.copy()
data_ref.loc[:, '참조조문'].fillna('None', inplace=True)

def call(raw_data):
    numberid = []
    for index, row in raw_data.iterrows():
        r = row["참조조문"].replace('\r', '').replace('\n', '').replace(" ", "").replace(",", " ")
        r = re.sub("군형법", "", r)
        numberid.append(r)
    return numberid

numberid = call(data_ref)



def criminal(numberid):
    numberid2 = []
    for sen in numberid:
        p = re.compile("(형법)+제+[0-9]+조+(\s+제+[0-9]+조)+|(형법)+제+[0-9]+조")
        index = [(m.start(), m.end()) for m in p.finditer(sen)]
        if len(index) == 0:
            numberid2.append(0)
        if len(index) > 0:
            law = []
            for i in index:
                law.append(sen[i[0]:i[1]])
            numberid2.append(law)
            law = []
    return numberid2

numberid2 = criminal(numberid)



def only_number(numberid2):
    number = []
    for sen in numberid2:
        p = re.compile("[0-9]+[0-9]+[0-9]|[0-9]+[0-9]|[0-9]")
        index = [(m.start(), m.end()) for m in p.finditer(str(sen))]
        if len(index) == 0:
            number.append([0])
        if len(index) > 0:
            law = []
            for i in index:
                law.append(str(sen)[i[0]:i[1]])
            law = list(set(int(x) for x in law))
            number.append(law)
            law = []
    return number

number = only_number(numberid2)
len(number)
data_label.shape
def assign_ref(data, number):
    for i, (index, row) in enumerate(data.iterrows()):
        for num in number[i]:
            if 87 <= num <= 91:
                data.loc[index, 1] = 1                
            elif 92<= num <= 104:
                data.loc[index, 2] = 1                
            elif 105<= num <= 106:
                data.loc[index, 3] = 1                
            elif 107<= num <= 113:
                data.loc[index, 4] = 1                
            elif 114<= num <= 118:
                data.loc[index, 5] = 1                
            elif 119<= num <= 121:
                data.loc[index, 6] = 1                
            elif 122<= num <= 135:
                data.loc[index, 7] = 1                
            elif 136<= num <= 144:
                data.loc[index, 8] = 1                
            elif 145<= num <= 151:
                data.loc[index, 9] = 1                
            elif 152<= num <= 155:
                data.loc[index, 10] = 1                
            elif 156<= num <= 157:
                data.loc[index, 11] = 1                
            elif 158<= num <= 163:
                data.loc[index, 12] = 1                
            elif 164<= num <= 176:
                data.loc[index, 13] = 1                
            elif 177<= num <= 184:
                data.loc[index, 14] = 1                
            elif 185<= num <= 191:
                data.loc[index, 15] = 1                
            elif 192<= num <= 197:
                data.loc[index, 16] = 1                
            elif 198<= num <= 206:
                data.loc[index, 17] = 1                
            elif 207<= num <= 213:
                data.loc[index, 18] = 1                
            elif 214<= num <= 224:
                data.loc[index, 19] = 1                
            elif 225<= num <= 237:
                data.loc[index, 20] = 1                
            elif 238<= num <= 240:
                data.loc[index, 21] = 1                
            elif 241<= num <= 245:
                data.loc[index, 22] = 1                
            elif 246<= num <= 249:
                data.loc[index, 23] = 1                
            elif 250<= num <= 256:
                data.loc[index, 24] = 1                
            elif 257<= num <= 265:
                data.loc[index, 25] = 1                
            elif 266<= num <= 268:
                data.loc[index, 26] = 1                
            elif 269<= num <= 270:
                data.loc[index, 27] = 1                
            elif 271<= num <= 275:
                data.loc[index, 28] = 1                
            elif 276<= num <= 282:
                data.loc[index, 29] = 1                
            elif 283<= num <= 286:
                data.loc[index, 30] = 1                
            elif 287<= num <= 296:
                data.loc[index, 31] = 1                
            elif 297<= num <= 306:
                data.loc[index, 32] = 1                
            elif 307<= num <= 312:
                data.loc[index, 33] = 1                
            elif 313<= num <= 315:
                data.loc[index, 34] = 1                
            elif 316<= num <= 318:
                data.loc[index, 35] = 1                
            elif 319<= num <= 322:
                data.loc[index, 36] = 1                
            elif 323<= num <= 328:
                data.loc[index, 37] = 1                
            elif 329<= num <= 346:
                data.loc[index, 38] = 1                
            elif 347<= num <= 354:
                data.loc[index, 39] = 1                
            elif 355<= num <= 361:
                data.loc[index, 40] = 1                
            elif 362<= num <= 365:
                data.loc[index, 41] = 1                
            elif 366<= num <= 372:
                data.loc[index, 42] = 1                
            else:
                pass
    return data
data_refd = assign_ref(data_ref, number)
# data_refd


In [7]:
total = []
for i in range(42):
    print(data_refd[data_refd[i+1] == 1].shape[0])
    total.append(data_refd[data_refd[i+1] == 1].shape[0])

print(sum(total))

16
152
1
4
34
10
1004
247
448
304
392
92
112
6
104
8
6
16
150
1407
31
168
121
540
1191
446
13
219
112
168
75
818
401
597
47
408
220
1526
2224
2208
146
222
16414


,사건명,참조조문,판결요지,판례내용,판시사항,1,2,3,4,5,...,33,34,35,36,37,38,39,40,41,42
prncYd,,,,,,,,,,,,,,,,,,,,,
73888,마약법위반피고사건,형사소송법 제298조,공동정범으로 공소가 제기된 경우 법원이 종범의 성립을 인정하였다 하더라도 심판의 범...,【피 고 인】 이정자\n【항 소 인】 피고인\n【원심판결】 \n제1심 전주지...,공동정범으로 공소가 제기된 경우 법원이 종범의 성립을 인정할 수 있는지 여부,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67312,공직선거및선거부정방지법위반,"[1]\n\n 공직선거및선거부정방지법 제58조 제1항\n,\n\n ...",NaN,【피고인】 \n【상고인】 검사\n【원심판결】 \n서울고법 2004. 1. 30....,[1] \n 공직선거및선거부정방지법 제93조 제1항에 의하여 금지되...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67316,노동조합및노동관계조정법위반,"[1]\n\n 노동조합및노동관계조정법 제81조 제3호\n,\n\n ...",NaN,【피고인】 \n【상고인】 검사\n【변호인】 법무법인 바른법률 담당변호사 조중...,(주)데이콤과 그 출자회사의 근로자를 조직대상으로 하는 데이콤노동조합이 규약을 개정...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67151,업무상과실치사,[1] \n 형사소송법 제308조 / [2] \n 형사소송...,NaN,【피고인】 \n【상고인】 검사\n【변호인】 변호사 최기만\n【원심판결】 \n...,[1] 형사재판에 있어서 자유심증주의의 한계 및 간접증거의 증명력\n[2] 수중시체...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67258,횡령,[1]\n\n 형법 제355조 제1항,NaN,【피고인】 \n【상고인】 피고인\n【변호인】 변호사 조명원\n【원심판결】 \...,수탁자가 할인을 위하여 교부받은 약속어음을 자신의 채무변제에 충당한 경우 횡령죄의 ...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:

with open('labeled_data.pkl', 'wb') as data:
    pickle.dump(data_refd, data, protocol=pickle.HIGHEST_PROTOCOL)
    

In [4]:
with open('labeled_data.pkl', 'rb') as data:
    data_refd = pickle.load(data)
data_refd.shape

(18189, 47)

In [24]:
data_refd.iloc[8]

사건명                                               살인·감호청구
참조조문    [1] \n        국민의 형사재판 참여에 관한 법률 제1조, \n      ...
판결요지    [1] 국민의 형사재판 참여에 관한 법률에 따라 권고적 효력에 그치고 있는 배심원의...
판례내용    【피고인 겸 피치료감호청구인】 \n【항 소 인】 피고인 및 검사\n【검    사】 ...
판시사항    [1] 국민의 형사재판 참여에 관한 법률에 따른 배심원의 평결이 실질적 효력을 갖도...
1                                                       0
2                                                       0
3                                                       0
4                                                       0
5                                                       0
6                                                       0
7                                                       0
8                                                       0
9                                                       0
10                                                      0
11                                                      0
12                                                      0
13            